<div style="display: flex; align-items: center; gap: 40px;">

<img src="https://play-lh.googleusercontent.com/_O9p4Z4yucA2NLmZBu9mTJCuBwXeT9NcbtrDN6I8gKlkIPRySV0adOmbyipjSj9Gew" width="138">
<img src="https://blog.langchain.dev/content/images/size/w990/format/webp/2024/04/Tool-Calling-2.png" width="185">





<div>
  <h2>SUTRA by TWO Platforms </h2>
  <p>SUTRA is a family of large multi-lingual language (LMLMs) models pioneered by Two Platforms. SUTRA’s dual-transformer approach extends the power of both MoE and Dense AI language model architectures, delivering cost-efficient multilingual capabilities for over 50+ languages. It powers scalable AI applications for conversation, search, and advanced reasoning, ensuring high-performance across diverse languages, domains and applications.</p>

  <h2>LangChain's Tool Calling</h2>
    <p>LangChain's tool calling enables language models to invoke external functions or APIs during conversations by defining tools with the @tool decorator and binding them to the model using .bind_tools(). This allows the model to generate structured outputs that match the tool's input schema, facilitating seamless integration of functionalities like calculations, translations, or data retrieval into AI applications.</p>
  </div>
</div>



[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/18POcDv67qN0_CJo-_BPr6VE-r5RlXUYC?usp=sharing)


## Get Your API Keys

Before you begin, make sure you have:

1. A SUTRA API key (Get yours at [TWO AI's SUTRA API page](https://www.two.ai/sutra/api))
2. Basic familiarity with Python and Jupyter notebooks

This notebook is designed to run in Google Colab, so no local Python installation is required.

##LangChain's Tool Calling With SUTRA

###1. Install Dependencies

In [ ]:
# Install required packages
!pip install -q langchain langchain-core langchain-openai requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 1.4 MB/s eta 0:00:00


### 2. Set Up Environment Variables

In [ ]:
import os
from google.colab import userdata

# Set the API key from Colab secrets
os.environ["SUTRA_API_KEY"] = userdata.get("SUTRA_API_KEY")

###3. Initialize SUTRA LLM

In [ ]:
from langchain_openai import ChatOpenAI

def get_sutra_llm(temperature=0.7):
    """Initialize Sutra LLM instance"""
    return ChatOpenAI(
        api_key=os.environ.get("SUTRA_API_KEY"),
        base_url="https://api.two.ai/v2",
        model="sutra-v2",
        temperature=temperature
    )

###4. Basic Math Tools

In [ ]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two integers and returns the result."""
    return a * b

# Call the tool manually to see the output
print(multiply.invoke({"a": 3, "b": 5}))


@tool
def add(a: int, b: int) -> int:
    """Adds two integers and returns the result."""
    return a + b

# Test call
print(add.invoke({"a": 10, "b": 15}))

15
25


###5. Language Detection & Translation Tools


In [ ]:
from langchain_core.tools import tool

@tool
def detect_language(text: str):
    """Detects the language of a given text based on predefined keywords."""
    languages = {
        "hello": "English", "hola": "Spanish", "namaste": "Hindi",
        "bonjour": "French", "ni hao": "Chinese", "konichiwa": "Japanese",
        "안녕하세요": "Korean"
    }
    text_lower = text.lower()
    detected = next((lang for key, lang in languages.items() if key in text_lower), "English")
    return {"detected_language": detected, "confidence": 0.9, "text": text}

# Test call
print(detect_language.invoke({"text": "Hola amigo"}))

@tool
def translate_text(text: str, source_language: str, target_language: str):
    """Translates text from source language to target language using predefined mappings."""
    translations = {
        ("English", "Spanish", "hello"): "hola",
        ("English", "Hindi", "hello"): "नमस्ते",
        ("Hindi", "English", "namaste"): "hello",
    }
    return translations.get((source_language, target_language, text.lower()), f"[{target_language}] {text}")

# Test call
print(translate_text.invoke({
    "text": "hello",
    "source_language": "English",
    "target_language": "Hindi"
}))


{'detected_language': 'Spanish', 'confidence': 0.9, 'text': 'Hola amigo'}
नमस्ते


###6. Currency Conversion Tools

In [ ]:
from langchain_core.tools import tool
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def convert_currency(base_currency_value: float, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """Converts a currency value using the given conversion rate."""
    return base_currency_value * conversion_rate

# Example manual call
print(convert_currency.invoke({"base_currency_value": 100, "conversion_rate": 83.5}))


8350.0


###7. Weather Information (Mock)

In [ ]:
from langchain_core.tools import tool

@tool
def get_weather(city: str, country_code: str):
    """Returns mock weather data for a given city and country code."""
    weather_data = {
        "new york:us": {"temp": 22, "condition": "sunny"},
        "london:uk": {"temp": 15, "condition": "rainy"},
        "mumbai:in": {"temp": 32, "condition": "humid"}
    }
    key = f"{city.lower()}:{country_code.lower()}"
    return weather_data.get(key, {"temp": 25, "condition": "unknown", "humidity": 50})

# Test call
print(get_weather.invoke({"city": "Mumbai", "country_code": "IN"}))

{'temp': 32, 'condition': 'humid'}


### **8. Using bind_tools method**

In [ ]:
from langchain_core.tools import tool

@tool
def add_numbers(a: int, b: int) -> int:
    """Add two integers and return the result."""
    return a + b

@tool
def detect_language(text: str) -> str:
    """Detects the language based on common greetings."""
    greetings = {
        "hello": "English", "hola": "Spanish", "namaste": "Hindi",
        "bonjour": "French", "ni hao": "Chinese", "konichiwa": "Japanese",
        "안녕하세요": "Korean"
    }
    text_lower = text.lower()
    for key, lang in greetings.items():
        if key in text_lower:
            return lang
    return "Unknown"

###Usage

In [ ]:
sutra_llm = get_sutra_llm()
llm_with_tools = sutra_llm.bind_tools([add_numbers, detect_language])

print(llm_with_tools.invoke("What is 45 plus 17?"))
print(llm_with_tools.invoke("Which language is this: 'Bonjour'?"))

content='45 plus 17 equals 62.' additional_kwargs={'refusal': None} response_metadata={'token_usage': None, 'model_name': 'sutra-v2', 'system_fingerprint': None, 'id': '6UmnAgLSqTNsTkCPPZxpQE', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--0799d28a-bf89-49a0-ac07-8d880acecc20-0'
content="'Bonjour' is French for 'Hello'." additional_kwargs={'refusal': None} response_metadata={'token_usage': None, 'model_name': 'sutra-v2', 'system_fingerprint': None, 'id': 'wqJrrR3ZdXaRWf2kaycRA6', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--9f050b52-349f-4d3a-972a-d89006123d38-0'
